Why have I created this repo? I like to read a lot; papers, blog posts, twitter threads, notebooks, you name it. When I read, i've been quite well at documenting this through my blog. The blog let's me sement my thoughts, improve retention, and log what i've read. it can be quite tedious work, and sometimes I simply don't have the time, or rather capacity to do it, but i've been quite good at it, at least up until a couple of months ago. now, even though I sometimes write quite extensive summaries or thoughts about papers, i've been pretty poor at looking back at my notes when i need to remember something in a paper i read, and i attribute this mainly to a lack of **indexing**. the process often looks like this: "ooh, i know i've read something about X in a paper" *scrolls through all [blog entries](https://leonericsson.github.io/indexer), not finding what i'm looking for immediately and just give up*. yes, i could just become more patient and keep looking, why not instead take this opportunity and build something! this is the motivation for this repo. 

my initial plan. before doing any valuable research, here's what i **think** is the way i want to solve this problem, including some questions that i yet don't know the answer to:

1. save the link to every piece of research content i've read, that i can recall and find (i've already got a decent chunk saved).
2. download the content.
    - should everything be standardized into a format?
    - do i need to be careful to exclude irrelevant information from the source?
    - how do i deal with twitter threads?
3. decide on an embedding model
    - what is a good context size? relates to how we plan to chunk a 20 page paper.
4. embed content
    - how do we deal with long format texts e.g. papers?
    - does the search engine need to rely on more than just a single embedding model?
5. quantize embeddings to int8
    - does this process impact the selected embedding model?
5. index the embeddings
6. re-ranker
7. search engine
     - how do we link the source content + source url to the search results? especially
     considering that we will surely have multiple embedding points from the same source content.
    
okay, this seems like a good place to start don't you think?

**comparing embedding models**

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# 1. Specify preffered dimensions
dimensions = 512

# 2. load model
model_id = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)

# For retrieval you need to pass this prompt.
query = 'Represent this sentence for searching relevant passages: are all layers in a transformer equally important?'

docs = [
    query,
    """Within the field of vector search, an intriguing development has arisen: binary vector search. This approach shows promise in tackling the long-standing issue of memory consumption by achieving a remarkable 30x reduction. However, a critical aspect that sparks debate is its effect on accuracy.

We believe that using binary vector search, along with specific optimization techniques, can maintain similar accuracy. To provide clarity on this subject, we showcase a series of experiments that will demonstrate the effects and implications of this approach.""",
    """We empirically study a simple layer-pruning strategy for popular families of openweight pretrained LLMs, finding minimal degradation of performance on different
question-answering benchmarks until after a large fraction (up to half) of the layers
are removed. To prune these models, we identify the optimal block of layers to
prune by considering similarity across layers; then, to “heal” the damage, we
perform a small amount of finetuning. In particular, we use parameter-efficient
finetuning (PEFT) methods, specifically quantization and Low Rank Adapters
(QLoRA), such that each of our experiments can be performed on a single A100
GPU. From a practical perspective, these results suggest that layer pruning methods
can complement other PEFT strategies to further reduce computational resources of
finetuning on the one hand, and can improve the memory and latency of inference
on the other hand. From a scientific perspective, the robustness of these LLMs
to the deletion of layers implies either that current pretraining methods are not
properly leveraging the parameters in the deeper layers of the network or that the
shallow layers play a critical role in storing knowledge.""",
    """In this work, we introduce a significant 1-bit LLM variant called BitNet b1.58, where every parameter
is ternary, taking on values of {-1, 0, 1}. We have added an additional value of 0 to the original 1-bit
BitNet, resulting in 1.58 bits in the binary system. BitNet b1.58 retains all the benefits of the original
1-bit BitNet, including its new computation paradigm, which requires almost no multiplication
operations for matrix multiplication and can be highly optimized. Additionally, it has the same energy
consumption as the original 1-bit BitNet and is much more efficient in terms of memory consumption,
throughput and latency compared to FP16 LLM baselines.""",
]

# 2. Encode
embeddings = model_id.encode(docs)

# Optional: Quantize the embeddings
binary_embeddings = quantize_embeddings(embeddings, precision="ubinary")

similarities = cos_sim(embeddings[0], embeddings[1:])
print('similarities:', similarities)


In [ ]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

matryoshka_dim = 512

model_id = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

sentences = [
    """search_document: Within the field of vector search, an intriguing development has arisen: binary vector search. This approach shows promise in tackling the long-standing issue of memory consumption by achieving a remarkable 30x reduction. However, a critical aspect that sparks debate is its effect on accuracy.

We believe that using binary vector search, along with specific optimization techniques, can maintain similar accuracy. To provide clarity on this subject, we showcase a series of experiments that will demonstrate the effects and implications of this approach.""",
    """search_document: We empirically study a simple layer-pruning strategy for popular families of openweight pretrained LLMs, finding minimal degradation of performance on different
question-answering benchmarks until after a large fraction (up to half) of the layers
are removed. To prune these models, we identify the optimal block of layers to
prune by considering similarity across layers; then, to “heal” the damage, we
perform a small amount of finetuning. In particular, we use parameter-efficient
finetuning (PEFT) methods, specifically quantization and Low Rank Adapters
(QLoRA), such that each of our experiments can be performed on a single A100
GPU. From a practical perspective, these results suggest that layer pruning methods
can complement other PEFT strategies to further reduce computational resources of
finetuning on the one hand, and can improve the memory and latency of inference
on the other hand. From a scientific perspective, the robustness of these LLMs
to the deletion of layers implies either that current pretraining methods are not
properly leveraging the parameters in the deeper layers of the network or that the
shallow layers play a critical role in storing knowledge.""",
    """search_document: In this work, we introduce a significant 1-bit LLM variant called BitNet b1.58, where every parameter
is ternary, taking on values of {-1, 0, 1}. We have added an additional value of 0 to the original 1-bit
BitNet, resulting in 1.58 bits in the binary system. BitNet b1.58 retains all the benefits of the original
1-bit BitNet, including its new computation paradigm, which requires almost no multiplication
operations for matrix multiplication and can be highly optimized. Additionally, it has the same energy
consumption as the original 1-bit BitNet and is much more efficient in terms of memory consumption,
throughput and latency compared to FP16 LLM baselines.""",
]
embeddings = model_id.encode(sentences, convert_to_tensor=True)
embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
#embeddings = embeddings[:, :matryoshka_dim]
#embeddings = F.normalize(embeddings, p=2, dim=1)

print(embeddings)


In [ ]:
query = "search_query: are all layers in a transformer equally important?"
query_embedding = model_id.encode(query, convert_to_tensor=True)

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

model_id = SentenceTransformer(
    "jinaai/jina-embeddings-v2-base-en", # switch to en/zh for English or Chinese
    trust_remote_code=True
)

# control your input sequence length up to 8192
model_id.max_seq_length = 1024

embeddings = model_id.encode([
    'How is the weather today?',
    'What is the current weather like today?'
])
print(cos_sim(embeddings[0], embeddings[1]))


I spent the last 30 minutes comparing embedding models before realizing that this doesn't matter right now. we'll come back to this decision when we have a framework to populate the embedding space, at this stage any decent model will do. i'll leave the links to the prospects for future reference

1. https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
2. https://huggingface.co/jinaai/jina-embeddings-v2-base-en
3. https://huggingface.co/nomic-ai/nomic-embed-text-v1.5

for no specific reason let's move forward with mixedbread's embedder for now..


---

now that we've settled on an embedding model, the next major consideration is how to handle the varying content lengths of our documents. i want to be very specific in my search queries, far more so than what embedding only the abstract of a paper allows. for instance, i know that the llama 2 paper contains detailed ablations comparing mqa, gqa, and mha; and i should easily be able to query for that. so what granularity of document chunking does this specificity require? i don't know. but we'll find out together! starting of with the most naive approach; just embed the entire document!

let's download and store our content

In [ ]:
import requests
import fitz  # PyMuPDF
from bs4 import BeautifulSoup
from typing import Tuple

def download(url: str) -> Tuple[str, str]:
    """Download content from the given URL, determine its type, and extract the title and text."""
    response = requests.get(url)
    content_type = response.headers['Content-Type']

    def _parse_html(content: bytes) -> Tuple[str, str]:
        """
        Extract the title and text content from HTML data.
        """
        soup = BeautifulSoup(content, 'html.parser')
        title = soup.find('title').text
        text = soup.get_text()
        
        text = text.replace('\n', ' ')
        text = ' '.join(text.split())
        
        return title, text
    
    def _parse_pdf(content: bytes) -> Tuple[str, str]:
        """
        Extract the title and text content from a PDF file.
        """
        filename = 'document.pdf'
        with open(filename, 'wb') as f:
            f.write(content)
        
        doc = fitz.open(filename)
        text = ""
        for page in doc:
            text += page.get_text()
        
        text = text.replace('\n', ' ')
        text = ' '.join(text.split())

        # Extract title from metadata or the first block of text
        metadata = doc.metadata
        title = metadata.get('title', '')
        if not title:
            first_page = doc[0]
            blocks = first_page.get_text("blocks")
            
            # Assuming the title is in the first block
            if blocks:
                title = blocks[0][4].strip()
            else:
                title = 'unknown'

        return title, text

    if 'text/html' in content_type:
        title, text = _parse_html(response.content)
    elif 'application/pdf' in content_type:
        title, text = _parse_pdf(response.content)
    else:
        raise Exception('Unsupported content type')
    
    # Save content to file
    with open(f'data/{title}.txt', 'w') as f:
        f.write(text)

    return title, text

the above is straightforward. most of our content will either be html parsable by beautifulsoup or a pdf, which we can handle using this [pymupdf](https://pymupdf.readthedocs.io/en/latest/the-basics.html) package i found. i’ve future-proofed by implementing download(url), which takes a url and downloads the content to a .txt file. we’ll revisit the save format in the future; perhaps some kind of json structure, given that we’ll want to link content chunks to urls. however, that is for future me to solve.

In [ ]:
download("https://arxiv.org/pdf/2401.01325")
download("https://arxiv.org/pdf/2310.02207")
download("https://huggingface.co/blog/moe#:~:text=Mixture%20of%20Experts%20enable%20models,budget%20as%20a%20dense%20model.")

i've downloaded three papers that i'm going to embed

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# 1. Specify preffered dimensions
dimensions = 512

# 2. load model
model_id = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)

query_prefix = 'Represent this sentence for searching relevant passages: '

In [ ]:
import os

# load all documents in the data folder, returning a list of text content
def load(dir: str) -> list[str]:
    data = []
    for filename in os.listdir(dir):
        with open(f'data/{filename}', 'r') as f:
            data.append(f.read())
    return data

In [ ]:
documents = load('data')
index = model_id.encode(documents)

In [ ]:
def search(query: str) -> list[str]:
    query = query_prefix + query
    query_embedding = model_id.encode(query)
    similarities = cos_sim(index, query_embedding)
    return similarities

we've got three papers embedded in our database. this is a list of specific concepts from one of these papers:

1. positional out-of-distribution
2. passkey retrieval
3. context extension without fine-tuning
4. RoPE
5. selfextend

In [ ]:
search_queries = ["positional out-of-distribution", "passkey retrieval",
                  "context extension without fine-tuning", "RoPE",
                  "selfextend"]

for sq in search_queries:
    print(f"Query: {sq}")
    results = search(sq)
    for i, r in enumerate(results):
        print(f"Document {i+1}: {r}")
    print()

The concepts are all extracted from Document 3. Note the wavering stability in similarity between the query and doc 3.

Let's see what happens if we split each document into N chunks, linking each chunk back to its source document. This should drastically increase similarity.

In [ ]:
def chunk(documents: list[str], N: int) -> Tuple[list[str], list[int]]:
    """ Split documents into 512-word segments. """
    prev_chunk_index = 0
    chunk_index = []
    document_chunks = []
    for _, doc in enumerate(documents):
        # chunk document into 512-word segments
        chunks = [doc[i:i+N] for i in range(0, len(doc), N)]
        if len(chunks[-1]) != N:
            chunks.pop()
        
        chunk_index.append(prev_chunk_index + len(chunks))
        document_chunks.extend(chunks)
        prev_chunk_index = chunk_index[-1]

    return document_chunks, chunk_index

Split the documents into fixed-size segments. For each document, create segments of a specified length, ensuring uniformity by discarding smaller, incomplete segments. Maintain a cumulative count of segments for each document and compile all segments into a single list. The function returns both the list of segments and the cumulative segment indices for tracking purposes.

In [ ]:
import numpy
import bisect

def search_top_k(query: str, top_k: int, n_chunks: list[int]) -> list[str]:
    """
    Given a query, return the top-k most relevant documents, by index in n_chunks.
    """
    query = query_prefix + query
    query_embedding = model_id.encode(query)
    similarities = cos_sim(index, query_embedding).numpy()
    top_k_indices = numpy.argsort(similarities)[:top_k]
    
    source_documents = []
    for ind in top_k_indices:
        doc = bisect.bisect_right(n_chunks, ind)
        source_documents.append(doc)

    return source_documents

i'm still uncertain on the implementation of search, it depends on what we want to display as results of a query. In it's current state, search will return the documents that match the top_k chunks in the embedding search, without filtering out multiple pointers to the same source.

In [ ]:
# static per db
documents = load('data')
new_chunks, n_chunks = chunk(documents, 512) 
index = model_id.encode(new_chunks)

In [ ]:
search_queries = ["positional out-of-distribution", "passkey retrieval",
                  "context extension without fine-tuning", "RoPE",
                  "selfextend"]

for sq in search_queries:
    print(f"Query: {sq}")
    result = search_top_k(sq, 3, n_chunks)
    print(f"Top-3 documents: {result}")
    print()

The results aren't surprising, but encouraging nonetheless. We need to remember that we're going to introduce a re-ranking algorithm around here as well. We'll re-rank on the chunked embeddings, then comes the question whether we want to completely disregard chunks that pertain to the same source document or not, this depends on what we want to display in the results of the search; just the source document or the best matching chunk(s) as well. 

you know what, on second thought, after re-ranking the embedding search results we could actually: (1) **sum** the similarity scores per source document or (2) count the number of pointers to the source document! This would improve the search, prioritizing documents that contain multiple matches to the query.

let's gather everything that we're using down below

In [ ]:
import numpy as np
import os

import requests
import fitz  # PyMuPDF
from bs4 import BeautifulSoup
from typing import Tuple, List

Array = np.ndarray

# Constants
CHUNK_SIZE = 512
DATA_DIR = 'data'

def download(url: str) -> Tuple[str, str]:
    """Download content from the given URL, determine its type, and extract the title and text."""
    response = requests.get(url)
    content_type = response.headers['Content-Type']

    if 'text/html' in content_type:
        title, text = _parse_html(response.content)
    elif 'application/pdf' in content_type:
        title, text = _parse_pdf(response.content)
    else:
        raise Exception('Unsupported content type')
    
    # Save content to file
    with open(f'{DATA_DIR}/{title}.txt', 'w') as f:
        f.write(text)

    return title, text

def _parse_html(content: bytes) -> Tuple[str, str]:
    """
    Extract the title and text content from HTML data.
    """
    soup = BeautifulSoup(content, 'html.parser')
    title = soup.find('title').text
    text = _clean_text(soup.get_text())
    
    return title, text
    
def _parse_pdf(content: bytes) -> Tuple[str, str]:
    """
    Extract the title and text content from a PDF file.
    """
    filename = 'temp.pdf'
    with open(filename, 'wb') as f:
        f.write(content)
    
    doc = fitz.open(filename)
    text = ' '.join([page.get_text() for page in doc])
    text = _clean_text(text)

    # Extract title from metadata or the first block of text
    title = doc.metadata.get('title', _extract_first_block_text(doc[0]))

    os.remove(filename)
    return title, text

def _clean_text(text: str) -> str:
    text = text.replace('\n', ' ')
    return ' '.join(text.split())

def _extract_first_block_text(page) -> str:
    blocks = page.get_text("blocks")
    return blocks[0][4].strip() if blocks else 'unknown'

def load(dir: str) -> list[str]:
    """ Load all documents in the data folder, returning a list of text content."""
    data = []
    for filename in os.listdir(dir):
        with open(f'{DATA_DIR}/{filename}', 'r') as f:
            data.append(f.read())
    return data

def chunk(documents: list[str], N: int = CHUNK_SIZE) -> Tuple[List[str], List[int]]:
    """ Split documents into 512-word segments. """
    prev_chunk_index = 0
    chunk_index = []
    document_chunks = []
    for _, doc in enumerate(documents):
        # chunk document into 512-word segments
        chunks = [doc[i:i+N] for i in range(0, len(doc), N)]
        if len(chunks[-1]) != N:
            chunks.pop()
        
        chunk_index.append(prev_chunk_index + len(chunks))
        document_chunks.extend(chunks)
        prev_chunk_index = chunk_index[-1]

    return document_chunks, chunk_index

def search_top_k(db_embedding: Array, query: List[str], top_k: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Given queries, find top-k document chunk matches by cosine similarity. Exact NN search.
    """
    query_embedding = model_id.encode(query_prefix + query)
    similarities = cos_sim(db_embedding, query_embedding).numpy()
    top_k_indices = np.argsort(similarities, axis=0, )[-top_k:][::-1]
    
    return top_k_indices, similarities[top_k_indices]

def aggregate_and_sort(documents: Array, scores: Array) -> Tuple[Array, Array]:
    """ Aggregate chunk similarity scores by source document and sort the documents by new scores."""

    unique_docs, inverse_indices = np.unique(documents, return_inverse=True)    
    aggregated_scores = np.bincount(inverse_indices, weights=scores)
    sorted_indices = np.argsort(-aggregated_scores)
    
    sorted_documents = unique_docs[sorted_indices]
    sorted_scores = aggregated_scores[sorted_indices]
    return sorted_documents, sorted_scores

def search(db_embedding: Array, queries: List[str], top_k: int, n_chunks: List[int]) -> List[int]:
    """
    Search the embedding database for the most relevant documents to the query. 
    """
    
    top_k_chunks, scores = search_top_k(db_embedding, queries, top_k)
    # re-rank the top-k document chunks

    flat_indices = top_k_chunks.flatten()
    flat_scores = scores.flatten()
    top_k_documents = np.searchsorted(n_chunks, flat_indices, side='right')
    top_k_documents = aggregate_and_sort(top_k_documents, flat_scores)

    return top_k_documents
    

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# 1. Specify preffered dimensions
dimensions = 512

# 2. load model
model_id = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)

query_prefix = 'Represent this sentence for searching relevant passages: '

# load data and initialize embedded db
documents = load('data')
new_chunks, n_chunks = chunk(documents, 512) 
index = model_id.encode(new_chunks)

this is everything we've got so far. i've cleaned up the search code, it now aggregates similarity scores pertaining to the same
source document. we also have easy access to the chunk matches if we need them down the line. it's using brute-force search atm, we should consider
swapping this to something more effective.

we're still missing a [re-ranking algorithm](https://huggingface.co/BAAI/bge-reranker-base). i'm yet unsure if this is actually necessary given the small embedding space we're working with. i'll leave this as an ablation
for much later.

In [ ]:
import faiss

def faiss(index, db_embedding: Array, query: str, top_k: int) -> Tuple[List[int], List[int]]:
    """
    Given a query, find top-k document chunk matches by cosine similarity. Exact NN search.
    """
    query = query_prefix + query
    query_embedding = model_id.encode(query)

    # Perform ANN search
    _, I = index.search(query_embedding[np.newaxis, :], top_k)
    I = I.flatten() # remove batch dimension
    top_k_vectors = db_embedding[I]
    similarities = cos_sim(query_embedding, top_k_vectors).numpy().flatten()

    return I, similarities

def search_faiss(index, db_embedding: Array, query: str, top_k: int, n_chunks: List[int]) -> List[int]:
    """
    Search the embedding database for the most relevant documents to the query. 
    """
    
    top_k_indices, scores = faiss(index, db_embedding, query, top_k)
    # re-rank the top-k document chunks
    top_k_documents = np.searchsorted(n_chunks, top_k_indices, side='right')
    top_k_documents = aggregate_and_sort(top_k_documents, scores)

    return top_k_documents

index = faiss.IndexFlatL2(index.shape[1])
index.add(index)

I implemented a FAISS version of the search and benchmarked it against our previous solution. Note the difference in `faiss()` vs `search_top_k()`; faiss uses a flat index and performs a optimized KNN search in the embedding space, while search_top_k calculates the cosine similarity for every vector in the embedding database and then sorts it using quicksort (`np.argsort`). Benchmarking the two against each other:

`%timeit search(db_embedding, "positional out-of-distribution", 50, n_chunks)`

50.86 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

`%timeit search_faiss(index, db_embedding, "positional out-of-distribution", 50, n_chunks)`

93.2 ms ± 6.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

naive brute force is quicker. remember that these libraries are implemented to **scale**. Right now, we don't have scale. but we can come back to this comparison later down the line.

---

let's take stock of what we've got so far. We can input a bunch of source urls, download the content, create a vector storage from chunking these documents, query this chunk space (top_k) and ultimately return the source document index of the best matching chunk.

Note that there are a couple of key data holders in this setup:

1. The embedding database `db_embedding`
2. The chunk -> document index `n_chunks`
3. The original documents `documents`
4. The chunks `chunks`

and because we need someone to own this data now is a good time to move our implementation to a class.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle
import requests
import tempfile
from typing import Union, Tuple, List

import numpy as np
import fitz  # PyMuPDF
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from vector_store import VectorStorage, SimilarityMetric

Array = np.ndarray

class Mindex:
    def __init__(self, name: str, model_id: str = "mixedbread-ai/mxbai-embed-large-v1", EMBEDDING_DIM: int = 512, CHUNK_SIZE: int = 200, QUERY_PREFIX = '') -> None:
        self.NAME = name
        self.CHUNK_SIZE = CHUNK_SIZE
        self.EMBEDDING_DIM = EMBEDDING_DIM
        self.model_id = model_id

        self.storage = VectorStorage(
            embedder=SentenceTransformer(model_id, truncate_dim=EMBEDDING_DIM),
            similarity=SimilarityMetric.COSINE,
            query_prefix=QUERY_PREFIX,
            save_embedder=False
        )


        self.documents: List[Tuple[str, str]] = []
        self.chunks: List[str] = []
        self.chunk_index: List[int] = [0]

    def add(self, urls: Union[Tuple[str, ...], List[str]] = [], filename: str = None):
        """Add document(s) to Mindex.

        Args:
            urls (Union[Tuple[str, ...], List[str]]): List of URLs to add.
            filename (str, optional): Path to a file containing URLs to add.

        
        """
        assert isinstance(urls, (tuple, list))
        assert urls != [] or filename is not None
        
        if filename:
            with open(filename, 'r') as f:
                urls.extend([line.strip() for line in f.readlines()])

        new_chunks = []
        for url in urls:
            if url in [doc[1] for doc in self.documents]:
                print(f"Skipped {url} as it already exists in the index.")
                continue

            title, text = self._download(url)
            self.documents.append((title, url))
            chunks, n_chunks = self._chunk(text)
            new_chunks.extend(chunks)
            self.chunk_index.append(self.chunk_index[-1] + n_chunks)

        assert new_chunks != []
        
        self.storage.index(new_chunks)
        self.chunks.extend(new_chunks)

        self.save(f"{self.NAME}.pkl")

    
    def save(self, filename: str):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)


    @classmethod
    def load(cls, filename: str):
        with open(filename, 'rb') as f:
            obj = pickle.load(f)
            
            # reload embedding model
            obj.storage.set_embedder(
                SentenceTransformer(
                    obj.model_id, 
                    truncate_dim=obj.EMBEDDING_DIM
                    )
                )
            return obj

    def search(self, query: str, top_k: int) -> Tuple[Array, Array, Array]:
        """
        Search the embedding database for the most relevant documents to the query.

        Args:
            query (str): The search query.
            top_k (int): The number of top results to return.

        Returns:
            Tuple[np.ndarray, np.ndarray, np.ndarray]: Top documents, doc scores, top chunks, chunk scores.
        """
        assert top_k > 0 and top_k <= len(self.chunks)
        
        top_k_chunks, chunk_scores = self.storage.search_top_k([query], top_k)
        top_k_chunks = top_k_chunks.squeeze()
        chunk_scores = chunk_scores.squeeze()

        # connect chunks to documents
        top_k_documents = np.searchsorted(self.chunk_index, top_k_chunks, side='right') 
        top_m_documents, document_scores = self._aggregate_and_sort(top_k_documents, chunk_scores) # m <= k

        return top_m_documents.squeeze(), document_scores.squeeze(), top_k_chunks, chunk_scores


    def _aggregate_and_sort(self, documents: Array, scores: Array) -> Tuple[Array, Array]:
        """ Aggregate chunk similarity scores by source document and sort the documents by new scores."""

        unique_docs, inverse_indices = np.unique(documents, return_inverse=True)    
        aggregated_scores = np.bincount(inverse_indices, weights=scores)
        sorted_indices = np.argsort(-aggregated_scores)
        
        sorted_documents = unique_docs[sorted_indices]
        sorted_scores = aggregated_scores[sorted_indices]
        return sorted_documents, sorted_scores
    

    def _chunk(self, text: str) -> Tuple[List[str], int]:
        """Split documents into 50% overlapping segments."""
        words = text.split()
        chunks = [' '.join(words[i:i+self.CHUNK_SIZE]) for i in range(0, len(words), self.CHUNK_SIZE // 2)]
        return chunks, len(chunks)

    
    def _download(self, url: str) -> Tuple[str, str]:
        """Download content from the given URL, determine its type, and extract the title and text."""
        response = requests.get(url)
        response.raise_for_status() 

        content_type = response.headers['Content-Type']

        if 'text/html' in content_type:
            return self._parse_html(response.content)
        elif 'application/pdf' in content_type:
            return self._parse_pdf(response.content)
        else:
            raise Exception('Unsupported content type')

    def _parse_html(self, content: bytes) -> Tuple[str, str]:
        """Extract the title and text content from HTML data."""
        soup = BeautifulSoup(content, 'html.parser')
        title = soup.find('title').text if soup.find('title') else 'No Title'
        text = self._clean_text(soup.get_text())
        return title, text

    def _parse_pdf(self, content: bytes) -> Tuple[str, str]:
        """Extract the title and text content from a PDF file."""
        with tempfile.NamedTemporaryFile(delete=False, suffix='.txt') as temp_file:
            temp_file.write(content)
            temp_file_path = temp_file.name

        doc = fitz.open(temp_file_path)
        text = ' '.join([page.get_text() for page in doc])
        text = self._clean_text(text)

        # Extract title from metadata or the first block of text
        title = doc.metadata.get('title', self._extract_first_block_text(doc[0]))

        os.remove(temp_file_path)
        return title, text

    def _clean_text(self, text: str) -> str:
        text = text.replace('\n', ' ')
        text = text.replace('- ', '')
        return ' '.join(text.split())


    def _extract_first_block_text(self, page) -> str:
        blocks = page.get_text("blocks")
        return blocks[0][4].strip() if blocks else 'unknown'    
    

In [ ]:
urls = [
    "https://arxiv.org/pdf/2401.01325",
    "https://arxiv.org/pdf/2310.02207",
    "https://huggingface.co/blog/moe#:~:text=Mixture%20of%20Experts%20enable%20models,budget%20as%20a%20dense%20model."
]

mindex = Mindex('test')
mindex.add(urls)

top_docs, scores_doc, top_chunks, chunk_scores = mindex.search("positional out-of-distribution", 10)

note that i've refactored the vector embedding database into it's own class. it encapsulates everything related to the embedding space, allowing Mindex to interact purely through textual via `index` and `search_top_k` functions. Mindex carries the logic for chunk <-> document.

function profiling

In [ ]:
import cProfile
import pstats

# Profile and save to a file
cProfile.run('mindex.add(urls)', 'profile_output.prof')

# Load and print stats
with open('profile_stats.txt', 'w') as f:
    p = pstats.Stats('profile_output.prof', stream=f)
    p.sort_stats('cumulative').print_stats()

# To read the stats file
with open('profile_stats.txt', 'r') as f:
    print(f.read())


## Evaluation

cool. we've got much of the basics in place. we can create a Mindex; add documents through urls; query; get top matching documents, chunks and their source url. 

the core of Mindex is of course `search`, and being most important, we'd like to improve its performance, right? we could just vibe it out - implement things we *think* will better retrieval accuracy and then empirically evaluate the improvement. but it would be fun if we could be more quantitative about it. i'm thinking we decide on some metric(s), develop a synthetic benchmark dataset, evaluate our baseline and try to improve from there. one metric for retrieval accuracy and one for retrieval speed should be enough.

-- Where to look for improvements --

- Chunking strategy
    - We now split into 50% overlapping chunks of size 200 words.
    - Shift Overlap
    - Split by paragraph or section

- Embedding strategy
    - Simply embedding entire chunks + query might not be sufficient, especially when our query is short, or very specific in a term we are looking for. May struggle when term matching is crucial.
    - Ensemble semantic search with keyword search. Semantic + BM25?

- Embedding model
    - mixedbread-ai/mxbai-embed-large-v1

- Chunk -> Document
    - We aggregate all the chunks scores per document into a document score.
    - Are there other ways?



first let's build an evaluator

In [ ]:
import json
from typing import List, Dict, Any
from nltk import ngrams

class Evaluator:
    def __init__(self, file_path: str):
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        self.dataset_name = data.get('dataset', '')
        self.validation_set = data.get('validation', [])
        self.test_set = data.get('test', [])


    def get_validation_set(self) -> List[str]:
        return [(item['query'], item['answer']['text']) for item in self.validation_set]


    def get_validation_queries(self) -> List[str]:
        return [item['query'] for item in self.validation_set]


    def get_test_queries(self) -> List[str]:
        return [item['query'] for item in self.test_set]


    def get_validation_item_by_id(self, item_id: str) -> Dict[str, Any]:
        return next((item for item in self.validation_set if item['id'] == item_id), None)


    def get_test_item_by_id(self, item_id: str) -> Dict[str, Any]:
        return next((item for item in self.test_set if item['id'] == item_id), None)


    def get_answer_for_query(self, query: str, dataset: str = 'validation') -> Dict[str, Any]:
        target_set = self.validation_set if dataset == 'validation' else self.test_set
        item = next((item for item in target_set if item['query'] == query), None)
        return item['answer'] if item else None


    @staticmethod
    def calculate_ngram_overlap_score(retrieved_chunk: str, true_answer: str, n: int = 5) -> float:
        def get_ngrams(t: str, n: int):
            return set(ngrams(t.lower().split(), n))

        retrieved_ngrams = get_ngrams(retrieved_chunk, n)
        answer_ngrams = get_ngrams(true_answer, n)
        
        overlap = len(retrieved_ngrams.intersection(answer_ngrams))
        return overlap / len(answer_ngrams)

    def evaluate_retrieval(self, answer: str, retrieved_chunks: List[str], threshold: float = 0.7) -> Dict[str, Any]:
        scores = [self.calculate_ngram_overlap_score(chunk, answer) for chunk in retrieved_chunks]
        
        max_score = max(scores)
        best_chunk_index = scores.index(max_score)
        is_answer_found = max_score >= threshold
        
        return {
            'scores': scores,
            'max_score': max_score,
            'best_chunk_index': best_chunk_index,
            'is_answer_found': is_answer_found
        }

dataset = Evaluator("../data/eval.json")

In [ ]:
# mindex = Mindex('selfextend_eval')

# mindex.add(urls=["https://arxiv.org/pdf/2401.01325"])
mindex = Mindex.load('../selfextend_eval.pkl')

evaluation main function

In [ ]:
num_samples = 0
num_correct = 0
sum_overlap = 0.0
for query, answer in dataset.get_validation_set():
    #print(f"Query: {query}")
    #print(f"Answer: {answer}")
    _, _, chunk_idxs, _ = mindex.search(query, top_k=5)
    chunks = [mindex.chunks[i] for i in chunk_idxs]
    result = dataset.evaluate_retrieval(answer, chunks)

    num_samples += 1
    num_correct += int(result['is_answer_found'])
    sum_overlap += result['max_score']
    #print(result['is_answer_found'], result['scores'], chunks[result['best_chunk_index']])
    #print("------")
    #print("")

print(f"Accuracy: {num_correct / num_samples:.4f} %")
print(f"Mean n-gram overlap: {sum_overlap / num_samples:.4f}")

This is the evaluation module. Initialized with an evaluation dataset `data/eval.json`, it retrieves validation/test samples and assesses Mindex search results. Two key aspects are noteworthy: (1) the method for determining if a retrieved chunk matches the answer, and (2) the metrics used to evaluate the entire top-k retrieval.

(1) Initially, exact string matching yielded poor results, with no retrieved chunks matching the expected answers. Debugging revealed slight formatting differences between chunks and answers, causing the evaluation system to incorrectly mark correct retrievals as failures. This discrepancy arose because the evaluation dataset was synthetically generated using Claude, rather than downloaded and parsed like the documents added to Mindex. To address this issue, we now use n-gram overlap (n=3) with a threshold of 0.7 to determine if a retrieved chunk contains the answer.


(2) We employ two metrics for evaluation:

a. Top-k accuracy: If any of the retrieved chunks contains the answer (as defined by the method in (1) with a 0.7 threshold), we mark it as an accurate retrieval with a score of 1; otherwise, it's 0. This metric provides a clear percentage of successful retrievals but depends on a threshold and lacks granularity.

b. Mean n-gram overlap score: We calculate the average of the best chunk's n-gram overlap score per retrieval. This metric addresses the limitations of the top-k accuracy metric by providing more nuanced results.

---

Now we've got all of the code needed to iteratively refine Mindex and actually measure improvement quantitatively. I've moved Mindex and the Evaluator into their own files (I'm getting tired of scrolling through this notebook), and now I just need to curate the evaluation dataset.

... ... ...

couple of hours later, I think we've got our dataset (sonnet 3.5 is goated)! 

you can see the final version of the prompt i used to generate the dataset in `data/claude_prompt.txt`. it took me some time to get things working properly. I wanted to ensure that queries were general. Take "What's the benefit of using structured input and output with LLMs?" as an example. There should be no reference to a specific paper, author, or method. This type of query is representative of how I want to interact with Mindex, making it a good evaluation sample. Unfortunately, Sonnet was adamant about not following such queries, often referencing "this paper" or asking about specific methods like "in SelfExtend how...". I allowed some of these queries to remain, but did my best to steer Sonnet away from them whenever possible. In the end we end up with 400+ samples, split into validation & test sets. 

with both the benchmark dataset and the code done, we're finally ready to benchmark the baseline and improve from there.

In [ ]:
%load_ext autoreload
%autoreload 2

create new mindex

In [ ]:
from mindex import Mindex
from evaluator import Evaluator

evaluator = Evaluator("../data/eval.json")
urls = evaluator.get_document_urls()

mindex = Mindex('eval')
mindex.add(urls=urls)


load mindex

In [ ]:
from mindex import Mindex
from evaluator import Evaluator

evaluator = Evaluator("../data/eval.json")
mindex = Mindex.load('eval.pkl')

#### CLI

Short side quests, I need a command line interface. This is far from my favorite part of this project, but I realize that you have to be able to use Mindex somehow, and a CLI seems like the most straight-forward way to do so. I was picturing something like this:

```
$ mindex create [name]
$ mindex search [query]
$ mindex add [url]
$ mindex remove [url]
```

so I got to work and created `cli.py`. I haven't created a CLI for a long time, and it's actually my first time working with [click](https://click.palletsprojects.com/en/8.1.x/) (i was using [typer](https://typer.tiangolo.com/) first but then I read that click is more lighweight) but the process was pretty seamless. Took me some time to set up the project structure correctly but we got there. Thing is, the shit is really slow... i mean look at this

![](assets/search_timings.png)

it takes 13 seconds, the culprit being the embedding model initialization. (not sure if i mentioned this) i'm not pickling the SentenceTransformer embedding model as part of the serialization (at least by default) because it took about a 1GB of space which felt like a lot. so instead the mindex load function initializes the embedding model from the model name

````
@classmethod
    def load2(cls, filename: str):
        with open(filename, 'rb') as f:
            obj = pickle.load(f)
            
            # reload embedding model
            obj.storage.set_embedder(
                SentenceTransformer(
                    obj.model_id, 
                    truncate_dim=obj.EMBEDDING_DIM
                    )
                )
            return obj
````

which apparently is fucking slow. i hadn't considered at the time that I would have to load on every search. if we do actually pickle the model things get a bit more reasonable

![](assets/search_timings2.png)

unfortunately increasing search time, in CLI world, is quite difficult. there are massive discrepancies in search times between run 1 and subsequent runs

````
Time taken for search 1: 0.7984 seconds
Time taken for search 2: 0.0358 seconds
Time taken for search 3: 0.0339 seconds
Time taken for search 4: 0.0343 seconds
Time taken for search 5: 0.0346 seconds
````

which can't be solved because every cli call is a new process.

#### Baseline !!!

In [ ]:
import time
from tqdm import tqdm

log = False

num_samples = 0
num_correct = 0
sum_overlap = 0.0
search_times = []
eval_times = []

eval_set = evaluator.get_validation_set() 

for query, answer in tqdm(eval_set, desc="Processing"):
    search_start = time.time()
    _, _, chunk_idxs, _ = mindex.search(query, top_k=5)
    search_end = time.time()
    search_times.append(search_end - search_start)

    chunks = [mindex.chunks[i] for i in chunk_idxs]

    eval_start = time.time()
    result = evaluator.evaluate_retrieval(answer, chunks)
    eval_end = time.time()
    eval_times.append(eval_end - eval_start)

    num_samples += 1
    num_correct += int(result['is_answer_found'])
    sum_overlap += result['max_score']

    if log:
        print(f"Query: {query}")
        print(f"Answer: {answer}")
        print(result['is_answer_found'], result['max_score'], chunks[result['best_chunk_index']])
        print("------")
        print("")

print(f"Accuracy: {num_correct / num_samples:.4f}")
print(f"Mean n-gram overlap: {sum_overlap / num_samples:.4f}")
print(f"Mean search time: {sum(search_times) / len(search_times):.4f} seconds")
print(f"Mean evaluation time: {sum(eval_times) / len(eval_times):.4f} seconds")

### Let's make this thing better

-- Where to look for improvements --

- Chunking strategy
    - We now split into 50% overlapping chunks of size 200 words.
    - Shift Overlap
    - Split by paragraph or section

- Embedding strategy
    - Simply embedding entire chunks + query might not be sufficient, especially when our query is short, or very specific in a term we are looking for. May struggle when term matching is crucial.
    - Ensemble semantic search with keyword search. Semantic + BM25?

- Embedding model
    - mixedbread-ai/mxbai-embed-large-v1

- Chunk -> Document
    - We aggregate all the chunks scores per document into a document score.
    - Are there other ways?



#### Chunk & Clean

Chunking is where I think we can find our first improvement. To remind the reader, `_chunk()` currently splits documents into 50% overlapping segments containing 200 words.

Because Mindex is meant to support a wide array of sources, defining a universal cleaning (or even chunking for that matter) strategy is difficult. Even if we focus on just arXiv papers, PDFs are notoriously unstandardized. I don't want to pour an immense amount of time into this, so I'd love for something simple, like splitting chunks on semantic boundaries (think paragraphs). Now you'd think this is as simple as finding a \n\n, but no. Not for pdfs :). I'm going to see if the pdf parser library i'm using (pymupdf) can help me out, otherwise we might have to call this improvement attempt a failure.

... (some time later) yeah, unfortunately there is no *simple* way to reliably extract sections or paragraphs. I'm keen to look into Vik Paruchuri's work @ https://github.com/VikParuchuri, I know he's created some amazing PDF analyzers, but I don't want to sink time into this right now, so we'll leave this for now.

In [1]:
from mindex import Mindex

mindex = Mindex('test')
mindex.add(urls=["https://arxiv.org/pdf/2401.01325"])

In [1]:
from mindex import Mindex

mindex = Mindex.load('test.pkl')

/Users/leonericsson/miniconda3/envs/mindex/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from mindex import Mindex

mindex = Mindex.load('test.pkl')

#mindex.add(urls=["https://arxiv.org/pdf/2401.01325"])
#r = mindex.search("positional out-of-distribution", top_k=5)

but there's still some things I want to experiment with, chunk_size and chunk_overlap. A simple grid search over the two should suffice.

In [ ]:
import os
import numpy as np
from mindex import Mindex
from mindex.evaluator import Evaluator

chunk_sizes = np.array([500, 600])
chunk_overlaps = np.array([0.4, 0.5, 0.6])

evaluator = Evaluator("data/eval.json")
urls = evaluator.get_document_urls()

chunk_size_grid, overlap_ratio_grid = np.meshgrid(chunk_sizes, chunk_overlaps)
chunk_overlap_grid = (chunk_size_grid * overlap_ratio_grid).astype(int)

chunk_sizes_flat = chunk_size_grid.flatten()
chunk_overlaps_flat = chunk_overlap_grid.flatten()

results = np.zeros((len(chunk_sizes_flat), 3))

# Perform grid search
for i, (chunk_size, chunk_overlap) in enumerate(zip(chunk_sizes_flat, chunk_overlaps_flat)):
    print(f"Evaluating: CHUNK_SIZE={chunk_size}, CHUNK_OVERLAP={chunk_overlap}")
    
    mindex = Mindex('eval', CHUNK_SIZE=int(chunk_size), CHUNK_OVERLAP=int(chunk_overlap))
    mindex.add(urls=urls)
    
    score = evaluator.evaluate_mindex(mindex, validation_set=True)
    results[i] = [chunk_size, chunk_overlap, score]
    
    os.remove('eval.pkl')
    del mindex

sorted_indices = np.argsort(results[:, 2])[::-1]
sorted_results = results[sorted_indices]

# Display results
print("\nGrid Search Results:")
print("CHUNK_SIZE | CHUNK_OVERLAP | Score")
print("-" * 35)
for chunk_size, chunk_overlap, score in sorted_results:
    print(f"{chunk_size:^10.0f} | {chunk_overlap:^13.0f} | {score:.4f}")

My first experiment was run with 

```
chunk_sizes = np.array([100, 200, 400, 800])
chunk_overlaps = np.array([0.2, 0.4, 0.6, 0.8])
```

and you can find the results in `data/benchmark_results.md`. larger chunk sizes generally perform better, with a sweet spot around 400 to 800 words. 40% to 60% overlap percentage typically yields the best results. The standout performer is the 800-word chunk size with 40% overlap, hitting nearly 64% accuracy. However, because our evaluation is determined by whether a retrieved chunk contains the answer *somewhere* in the chunk, there is a bias towards systems with larger chunk size. I'm going to run a focus grid search in the 400 - 800, 40-60% span.


```
chunk_sizes = np.array([500, 600])
chunk_overlaps = np.array([0.4, 0.5, 0.6])
```

again we see that larger chunk sizes and larger overlap improves performance. I'm thinking 600 words with a 360 word overlap is a good tradeoff. Thats a 16 point improvement over the baseline(!?).

In [ ]:
from mindex import Mindex
from mindex.evaluator import Evaluator

evaluator = Evaluator("data/eval.json")
urls = evaluator.get_document_urls()

mindex = Mindex('eval', CHUNK_SIZE=600, CHUNK_OVERLAP=360)
mindex.add(urls=urls)

load mindex

In [61]:
from mindex import Mindex
from mindex.evaluator import Evaluator

evaluator = Evaluator("data/eval.json")
mindex = Mindex.load('eval.pkl')

In [74]:
title, text = mindex._download("https://huggingface.co/blog/mlabonne/abliteration")

Final notes (for future self):

- PDF Analyzers [surya](https://github.com/VikParuchuri/surya) could be very interesting. See more from [Vik](https://github.com/VikParuchuri?tab=repositories)
- Parser + chunker for TeX. TeX is a lot more structured than PDFs, and all arXiv papers have a TeX source. 

https://x.com/JinaAI_/status/1823756993108304135